<a href="https://colab.research.google.com/github/siting1206/NLP_HW2/blob/main/NLP_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install pandas nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.5 MB/s 
     |████████████████████████████████| 182 kB 57.8 MB/s 
     |████████████████████████████████| 7.6 MB 37.6 MB/s 


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import math
import numpy as np
import time
import torch, pandas as pd
import nltk
import re
nltk.download('punkt')

from transformers import set_seed
set_seed(123)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [82]:
data = pd.read_csv('/content/drive/MyDrive/NLP_assignment2/Dataset/train_ext.txt', sep='\t')

In [83]:
data.head()

,snippet,question,answer
0,"lebowski sweater replica jun 17 , 2013 history...","last 8 years life , galileo house arrest espou...",copernicus
1,"images knickerless women jun 17 , 2013 history...","last 8 years life , galileo house arrest espou...",copernicus
2,"christa mcauliffe autopsy jun 17 , 2013 histor...","last 8 years life , galileo house arrest espou...",copernicus
3,"assignment 13 nov 30 , 2014 2 1912 olympian fo...",2 1912 olympian football star carlisle indian ...,jim thorpe
4,amazon web services put records accepts keywor...,2 1912 olympian football star carlisle indian ...,jim thorpe


In [84]:
data = data.drop(columns=['question'])
data.head()

,snippet,answer
0,"lebowski sweater replica jun 17 , 2013 history...",copernicus
1,"images knickerless women jun 17 , 2013 history...",copernicus
2,"christa mcauliffe autopsy jun 17 , 2013 histor...",copernicus
3,"assignment 13 nov 30 , 2014 2 1912 olympian fo...",jim thorpe
4,amazon web services put records accepts keywor...,jim thorpe


In [87]:
data['ans_start'] = [y.index(x) if str(x) in y else 0 for x,y in zip(data["answer"],data["snippet"])]
data['ans_end'] = [x+len(y)-1 if x!=0 else 0 for x,y in zip(data["ans_start"],data["answer"])]

In [88]:
data.head()

,snippet,answer,ans_start,ans_end
0,"lebowski sweater replica jun 17 , 2013 history...",copernicus,112,121
1,"images knickerless women jun 17 , 2013 history...",copernicus,112,121
2,"christa mcauliffe autopsy jun 17 , 2013 histor...",copernicus,113,122
3,"assignment 13 nov 30 , 2014 2 1912 olympian fo...",jim thorpe,0,0
4,amazon web services put records accepts keywor...,jim thorpe,0,0
